In [ ]:
!pip install transformers
!pip install keras
!pip install Sentencepiece
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install datasets
!pip install -U scikit-learn

In [1]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["WANDB_DISABLED"] = "true"

In [2]:
import transformers
import torch
from datasets import load_dataset, concatenate_datasets, Dataset, DatasetDict

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from functions import *

from transformers import TextClassificationPipeline
from transformers.pipelines.pt_utils import KeyDataset

from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from sklearn.model_selection import StratifiedKFold
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer

2022-10-23 14:25:25.816677: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


# Valence

In [4]:
label_num = 2
conf_scores = [0.5, 0.7, 0.95]
conf_scores_lower = [0.4, 0.3, 0.2, 0.1]
#load_tokenizer("distilbert-base-german-cased")
folds = StratifiedKFold(n_splits=10)

forum = load_tokenize_enhanced_set("../Ablation_Studies/forum/valence/train_valence_preproc.csv")
splits = folds.split(np.zeros(forum.num_rows), forum["labels"])

Using custom data configuration default-1336a50e6ad1460e
Found cached dataset csv (/home/jovyan/.cache/huggingface/datasets/csv/default-1336a50e6ad1460e/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/csv/default-1336a50e6ad1460e/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-9f90d038b32f01c6.arrow


In [ ]:
i = 0
    
training_args = TrainingArguments("original_forum", evaluation_strategy="epoch")
metrics = {}
for train_idxs, val_idxs in splits:
    fold_dataset = DatasetDict({
        "train":forum.select(train_idxs),
        "validation":forum.select(val_idxs)
    })
    print(i)
    model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-cased", num_labels=label_num)
    trainer = Trainer(model=model, args=training_args, train_dataset=fold_dataset["train"], eval_dataset=fold_dataset["validation"], compute_metrics=compute_metrics)
    trainer.train()
    metrics[i] = {}
    trainer.save_model("classif/stratified_cv_valence")
    metrics[i]["orig"] = compute_test_metrics(trainer.predict(fold_dataset["validation"]), None)
    metrics[i]["train_support"] = compute_test_metrics(trainer.predict(fold_dataset["train"]), None)
    model = AutoModelForSequenceClassification.from_pretrained("classif/stratified_cv_valence", num_labels=label_num)
    pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)
    dataset_1 = predict_valence_with_confidence("mot_05.csv", pipe)
    dataset_2 = predict_valence_with_confidence("mot_07.csv", pipe)
    dataset_3 = predict_valence_with_confidence("mot_095.csv", pipe)
    dataset_4 = predict_valence_with_confidence("new_data_both.csv", pipe)
    
    for conf_score in conf_scores:
        print(conf_score)
        evaluation, support = finetune_more(dataset_1, conf_score, fold_dataset["train"], 2, fold_dataset["validation"], training_args, mode="confidence")
        metrics[i]["FN 05 " + str(conf_score)] = evaluation
        metrics[i]["FN 05 " + str(conf_score)+"_support"] = support
        
        evaluation, support = finetune_more(dataset_2, conf_score, fold_dataset["train"], 2, fold_dataset["validation"], training_args, mode="confidence")
        metrics[i]["FN 07 " + str(conf_score)] = evaluation
        metrics[i]["FN 07 " + str(conf_score)+"_support"] = support
        
        evaluation, support = finetune_more(dataset_3, conf_score, fold_dataset["train"], 2, fold_dataset["validation"], training_args, mode="confidence")
        metrics[i]["FN 095 " + str(conf_score)] = evaluation
        metrics[i]["FN 095 " + str(conf_score)+"_support"] = support
        
        evaluation, support = finetune_more(dataset_4, conf_score, fold_dataset["train"], 2, fold_dataset["validation"], training_args, mode="confidence")
        metrics[i]["no FN " + str(conf_score)] = evaluation
        metrics[i]["no FN " + str(conf_score)+"_support"] = support
        
        metrics_df = pd.DataFrame.from_dict({(i,j): metrics[i][j]
                        for i in metrics.keys() 
                        for j in metrics[i].keys()},
                    orient='columns').transpose()
        metrics_df.to_csv("metrics/stratified_valence_FN_mot_test_sets_metrics.csv")
        i = i+1

### Choose Option once we have right results on stratified testing

In [ ]:
i = 0
conf_scores = [0.5]
    
training_args = TrainingArguments("original_forum", evaluation_strategy="epoch")
metrics = {}
for train_idxs, val_idxs in splits:
    fold_dataset = DatasetDict({
        "train":forum.select(train_idxs),
        "validation":forum.select(val_idxs)
    })
    print(i)
    if i > 8:
        metrics_old = pd.read_csv("metrics/stratified_valence_additional_tests.csv")
        model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-cased", num_labels=label_num)
        trainer = Trainer(model=model, args=training_args, train_dataset=fold_dataset["train"], eval_dataset=fold_dataset["validation"], compute_metrics=compute_metrics)
        trainer.train()
        metrics[i] = {}
        trainer.save_model("classif/stratified_cv_valence")
        metrics[i]["orig"] = compute_test_metrics(trainer.predict(fold_dataset["validation"]), None)
        metrics[i]["train_support"] = compute_test_metrics(trainer.predict(fold_dataset["train"]), None)
        model = AutoModelForSequenceClassification.from_pretrained("classif/stratified_cv_valence", num_labels=label_num)
        pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)
        '''dataset_1 = predict_valence_with_confidence("mot_05.csv", pipe)
        dataset_2 = predict_valence_with_confidence("mot_07.csv", pipe)
        dataset_3 = predict_valence_with_confidence("mot_095.csv", pipe)'''
        dataset_4 = predict_valence_with_confidence("new_data_both.csv", pipe)
        for conf_score in conf_scores:
            '''evaluation, support = finetune_more(dataset_1, conf_score, fold_dataset["train"], 2, fold_dataset["validation"], training_args)
            metrics[i]["FN 05 " + str(conf_score) + "_only_minority"] = evaluation
            metrics[i]["FN 05 " + str(conf_score)+"_support_only_minority"] = support

            evaluation, support = finetune_more(dataset_2, conf_score, fold_dataset["train"], 2, fold_dataset["validation"], training_args)
            metrics[i]["FN 07 " + str(conf_score)+ "_only_minority"] = evaluation
            metrics[i]["FN 07 " + str(conf_score)+"_support_only_minority"] = support

            evaluation, support = finetune_more(dataset_3, conf_score, fold_dataset["train"], 2, fold_dataset["validation"], training_args)
            metrics[i]["FN 095 " + str(conf_score) + "_only_minority"] = evaluation
            metrics[i]["FN 095 " + str(conf_score)+"_support_only_minority"] = support'''

            evaluation, support = finetune_more(dataset_4, conf_score, fold_dataset["train"], 2, fold_dataset["validation"], training_args)
            metrics[i]["no FN " + str(conf_score)+ "_only_minority"] = evaluation
            metrics[i]["no FN " + str(conf_score)+"_support_only_minority"] = support

            metrics_df = pd.DataFrame.from_dict({(i,j): metrics[i][j]
                            for i in metrics.keys() 
                            for j in metrics[i].keys()},
                        orient='columns').transpose()
            metrics_df = pd.concat([metrics_old, metrics_df], ignore_index=True)
            metrics_df.to_csv("metrics/stratified_valence_additional_tests.csv")

            '''evaluation, support = finetune_more(dataset_1, conf_score, fold_dataset["train"], 2, fold_dataset["validation"], training_args, mode="equal")
            metrics[i]["FN 05 " + str(conf_score) + "_equal"] = evaluation
            metrics[i]["FN 05 " + str(conf_score)+"_support_equal"] = support

            evaluation, support = finetune_more(dataset_2, conf_score, fold_dataset["train"], 2, fold_dataset["validation"], training_args, mode="equal")
            metrics[i]["FN 07 " + str(conf_score)+ "_equal"] = evaluation
            metrics[i]["FN 07 " + str(conf_score)+"_support_equal"] = support

            evaluation, support = finetune_more(dataset_3, conf_score, fold_dataset["train"], 2, fold_dataset["validation"], training_args, mode="equal")
            metrics[i]["FN 095 " + str(conf_score) + "_equal"] = evaluation
            metrics[i]["FN 095 " + str(conf_score)+"_support_equal"] = support'''

            evaluation, support = finetune_more(dataset_4, conf_score, fold_dataset["train"], 2, fold_dataset["validation"], training_args, mode="equal")
            metrics[i]["no FN " + str(conf_score)+ "_equal"] = evaluation
            metrics[i]["no FN " + str(conf_score)+"_support_equal"] = support

            metrics_df = pd.DataFrame.from_dict({(i,j): metrics[i][j]
                            for i in metrics.keys() 
                            for j in metrics[i].keys()},
                        orient='columns').transpose()
            #metrics_df.head()
            metrics_df = pd.concat([metrics_old, metrics_df], ignore_index=True)
            metrics_df.to_csv("metrics/stratified_valence_additional_tests.csv")

        for conf_score in conf_scores_lower:
            print(conf_score)

            '''evaluation, support = finetune_more(dataset_1, conf_score, fold_dataset["train"], 2, fold_dataset["validation"], training_args, mode="confidence")
            metrics[i]["FN 05 " + str(conf_score) + "_low_conf_minority"] = evaluation
            metrics[i]["FN 05 " + str(conf_score)+"_support_low_conf_minority"] = support

            evaluation, support = finetune_more(dataset_2, conf_score, fold_dataset["train"], 2, fold_dataset["validation"], training_args, mode="confidence")
            metrics[i]["FN 07 " + str(conf_score)+ "_low_conf_minority"] = evaluation
            metrics[i]["FN 07 " + str(conf_score)+"_support_low_conf_minority"] = support

            evaluation, support = finetune_more(dataset_3, conf_score, fold_dataset["train"], 2, fold_dataset["validation"], training_args, mode="confidence")
            metrics[i]["FN 095 " + str(conf_score) + "_low_conf_minority"] = evaluation
            metrics[i]["FN 095 " + str(conf_score)+"_support_low_conf_minority"] = support'''

            evaluation, support = finetune_more(dataset_4, conf_score, fold_dataset["train"], 2, fold_dataset["validation"], training_args, mode="confidence")
            metrics[i]["no FN " + str(conf_score)+ "_low_conf_minority"] = evaluation
            metrics[i]["no FN " + str(conf_score)+"_support_low_conf_minority"] = support

            metrics_df = pd.DataFrame.from_dict({(i,j): metrics[i][j]
                            for i in metrics.keys() 
                            for j in metrics[i].keys()},
                        orient='columns').transpose()
            #metrics_df.head()
            metrics_df = pd.concat([metrics_old, metrics_df], ignore_index=True)
            metrics_df.to_csv("metrics/stratified_valence_additional_tests.csv")
    i = i+1

# Labels

In [4]:
label_num = 3
conf_scores = [0.5, 0.7, 0.95]
conf_scores_lower = [0.4, 0.3, 0.2, 0.1]
folds = StratifiedKFold(n_splits=10)

forum = load_tokenize_enhanced_set("../Ablation_Studies/forum/label/train_label_preproc.csv")
splits = folds.split(np.zeros(forum.num_rows), forum["labels"])

Using custom data configuration default-097cb2dc197c9709
Found cached dataset csv (/home/jovyan/.cache/huggingface/datasets/csv/default-097cb2dc197c9709/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
i = 0
label_num = 3
    
training_args = TrainingArguments("original_forum", evaluation_strategy="epoch")
metrics = {}
for train_idxs, val_idxs in splits:
    fold_dataset = DatasetDict({
        "train":forum.select(train_idxs),
        "validation":forum.select(val_idxs)
    })
    print(i)
    model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-cased", num_labels=label_num)
    trainer = Trainer(model=model, args=training_args, train_dataset=fold_dataset["train"], eval_dataset=fold_dataset["validation"], compute_metrics=compute_metrics)
    trainer.train()
    metrics[i] = {}
    trainer.save_model("classif/stratified_cv_labels")
    metrics[i]["orig"] = compute_test_metrics(trainer.predict(fold_dataset["validation"]), None)
    metrics[i]["train_support"] = compute_test_metrics(trainer.predict(fold_dataset["train"]), None)
    model = AutoModelForSequenceClassification.from_pretrained("classif/stratified_cv_labels", num_labels=label_num)
    pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)
    dataset_1 = predict_labels_with_confidence("mot_05.csv", pipe)
    dataset_2 = predict_labels_with_confidence("mot_07.csv", pipe)
    dataset_3 = predict_labels_with_confidence("mot_095.csv", pipe)
    dataset_4 = predict_labels_with_confidence("new_data_both.csv", pipe)
    
    for conf_score in conf_scores:
        print(conf_score)
        evaluation, support = finetune_more_labels(dataset_1, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args, mode="confidence")
        metrics[i]["FN 05 " + str(conf_score)] = evaluation
        metrics[i]["FN 05 " + str(conf_score)+"_support"] = support
        
        evaluation, support = finetune_more_labels(dataset_2, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args, mode="confidence")
        metrics[i]["FN 07 " + str(conf_score)] = evaluation
        metrics[i]["FN 07 " + str(conf_score)+"_support"] = support
        
        evaluation, support = finetune_more_labels(dataset_3, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args, mode="confidence")
        metrics[i]["FN 095 " + str(conf_score)] = evaluation
        metrics[i]["FN 095 " + str(conf_score)+"_support"] = support
        
        evaluation, support = finetune_more_labels(dataset_4, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args, mode="confidence")
        metrics[i]["no FN " + str(conf_score)] = evaluation
        metrics[i]["no FN " + str(conf_score)+"_support"] = support
        
        metrics_df = pd.DataFrame.from_dict({(i,j): metrics[i][j]
                        for i in metrics.keys() 
                        for j in metrics[i].keys()},
                    orient='columns').transpose()
        metrics_df.to_csv("metrics/stratified_labels_FN_mot_test_sets_metrics.csv")
        
    i = i+1

### Choose Option once we have right results on stratified testing

In [ ]:
i = 0
label_num = 3
conf_scores = [0.95]
    
training_args = TrainingArguments("original_forum", evaluation_strategy="epoch")
metrics = {}
for train_idxs, val_idxs in splits:
    fold_dataset = DatasetDict({
        "train":forum.select(train_idxs),
        "validation":forum.select(val_idxs)
    })
    print(i)
    if i > 8:
        metrics_old = pd.read_csv("metrics/stratified_labels_additional_tests.csv")
        model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-cased", num_labels=label_num)
        trainer = Trainer(model=model, args=training_args, train_dataset=fold_dataset["train"], eval_dataset=fold_dataset["validation"], compute_metrics=compute_metrics)
        trainer.train()
        metrics[i] = {}
        trainer.save_model("classif/stratified_cv_labels")
        metrics[i]["orig"] = compute_test_metrics(trainer.predict(fold_dataset["validation"]), None)
        metrics[i]["train_support"] = compute_test_metrics(trainer.predict(fold_dataset["train"]), None)
        model = AutoModelForSequenceClassification.from_pretrained("classif/stratified_cv_labels", num_labels=label_num)
        pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)
        '''dataset_1 = predict_labels_with_confidence("mot_05.csv", pipe)
        dataset_2 = predict_labels_with_confidence("mot_07.csv", pipe)
        dataset_3 = predict_labels_with_confidence("mot_095.csv", pipe)'''
        dataset_4 = predict_labels_with_confidence("new_data_both.csv", pipe)

        for conf_score in conf_scores:
            print(conf_score)
            '''evaluation, support = finetune_more_labels(dataset_1, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args)
            metrics[i]["FN 05 " + str(conf_score) + "_only_minority"] = evaluation
            metrics[i]["FN 05 " + str(conf_score)+"_support_only_minority"] = support

            evaluation, support = finetune_more_labels(dataset_2, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args)
            metrics[i]["FN 07 " + str(conf_score)+ "_only_minority"] = evaluation
            metrics[i]["FN 07 " + str(conf_score)+"_support_only_minority"] = support

            evaluation, support = finetune_more_labels(dataset_3, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args)
            metrics[i]["FN 095 " + str(conf_score) + "_only_minority"] = evaluation
            metrics[i]["FN 095 " + str(conf_score)+"_support_only_minority"] = support'''

            evaluation, support = finetune_more_labels(dataset_4, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args)
            metrics[i]["no FN " + str(conf_score)+ "_only_minority"] = evaluation
            metrics[i]["no FN " + str(conf_score)+"_support_only_minority"] = support

            metrics_df = pd.DataFrame.from_dict({(i,j): metrics[i][j]
                            for i in metrics.keys() 
                            for j in metrics[i].keys()},
                        orient='columns').transpose()
            metrics_df = pd.concat([metrics_old, metrics_df], ignore_index=True)
            metrics_df.to_csv("metrics/stratified_labels_additional_tests.csv")

            '''evaluation, support = finetune_more_labels(dataset_1, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args, mode="equal")
            metrics[i]["FN 05 " + str(conf_score) + "_equal"] = evaluation
            metrics[i]["FN 05 " + str(conf_score)+"_support_equal"] = support

            evaluation, support = finetune_more_labels(dataset_2, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args, mode="equal")
            metrics[i]["FN 07 " + str(conf_score)+ "_equal"] = evaluation
            metrics[i]["FN 07 " + str(conf_score)+"_support_equal"] = support

            evaluation, support = finetune_more_labels(dataset_3, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args, mode="equal")
            metrics[i]["FN 095 " + str(conf_score) + "_equal"] = evaluation
            metrics[i]["FN 095 " + str(conf_score)+"_support_equal"] = support'''

            evaluation, support = finetune_more_labels(dataset_4, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args, mode="equal")
            metrics[i]["no FN " + str(conf_score)+ "_equal"] = evaluation
            metrics[i]["no FN " + str(conf_score)+"_support_equal"] = support

            metrics_df = pd.DataFrame.from_dict({(i,j): metrics[i][j]
                            for i in metrics.keys() 
                            for j in metrics[i].keys()},
                        orient='columns').transpose()
            #metrics_df.head()
            metrics_df = pd.concat([metrics_old, metrics_df], ignore_index=True)
            metrics_df.to_csv("metrics/stratified_labels_additional_tests.csv")

        for conf_score in conf_scores_lower:
            print(conf_score)

            '''evaluation, support = finetune_more_labels(dataset_1, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args, mode="confidence")
            metrics[i]["FN 05 " + str(conf_score) + "_low_conf_minority"] = evaluation
            metrics[i]["FN 05 " + str(conf_score)+"_support_low_conf_minority"] = support

            evaluation, support = finetune_more_labels(dataset_2, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args, mode="confidence")
            metrics[i]["FN 07 " + str(conf_score)+ "_low_conf_minority"] = evaluation
            metrics[i]["FN 07 " + str(conf_score)+"_support_low_conf_minority"] = support

            evaluation, support = finetune_more_labels(dataset_3, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args, mode="confidence")
            metrics[i]["FN 095 " + str(conf_score) + "_low_conf_minority"] = evaluation
            metrics[i]["FN 095 " + str(conf_score)+"_support_low_conf_minority"] = support'''

            evaluation, support = finetune_more_labels(dataset_4, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args, mode="confidence")
            metrics[i]["no FN " + str(conf_score)+ "_low_conf_minority"] = evaluation
            metrics[i]["no FN " + str(conf_score)+"_support_low_conf_minority"] = support

            metrics_df = pd.DataFrame.from_dict({(i,j): metrics[i][j]
                            for i in metrics.keys() 
                            for j in metrics[i].keys()},
                        orient='columns').transpose()
            #metrics_df.head()
            metrics_df = pd.concat([metrics_old, metrics_df], ignore_index=True)
            metrics_df.to_csv("metrics/stratified_labels_additional_tests.csv")
    i = i+1

# Sublabels

In [3]:
label_num = 4
conf_scores = [0.5, 0.7, 0.95]
conf_scores_lower = [0.4, 0.3, 0.2, 0.1]
#load_tokenizer("distilbert-base-german-cased")
folds = StratifiedKFold(n_splits=10)

forum = load_tokenize_enhanced_set("../Ablation_Studies/forum/sublabel/train_sublabel_preproc.csv")
splits = folds.split(np.zeros(forum.num_rows), forum["labels"])

Using custom data configuration default-3edc0ad1ac61e499
Found cached dataset csv (/home/jovyan/.cache/huggingface/datasets/csv/default-3edc0ad1ac61e499/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/csv/default-3edc0ad1ac61e499/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-e1d8785294ecdfce.arrow


In [8]:
i = 0
label_num = 4

training_args = TrainingArguments("original_forum", evaluation_strategy="epoch")
metrics = {}
for train_idxs, val_idxs in splits:
    fold_dataset = DatasetDict({
        "train":forum.select(train_idxs),
        "validation":forum.select(val_idxs)
    })
    print(i)
    model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-cased", num_labels=label_num)
    trainer = Trainer(model=model, args=training_args, train_dataset=fold_dataset["train"], eval_dataset=fold_dataset["validation"], compute_metrics=compute_metrics)
    trainer.train()
    metrics[i] = {}
    trainer.save_model("classif/stratified_cv_sublabels")
    metrics[i]["orig"] = compute_test_metrics(trainer.predict(fold_dataset["validation"]), None)
    metrics[i]["train_support"] = compute_test_metrics(trainer.predict(fold_dataset["train"]), None)
    model = AutoModelForSequenceClassification.from_pretrained("classif/stratified_cv_sublabels", num_labels=label_num)
    pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)
    dataset_1 = predict_sublabels_with_confidence("mot_05.csv", pipe)
    dataset_2 = predict_sublabels_with_confidence("mot_07.csv", pipe)
    dataset_3 = predict_sublabels_with_confidence("mot_095.csv", pipe)
    dataset_4 = predict_sublabels_with_confidence("new_data_both.csv", pipe)
    
    for conf_score in conf_scores:
        print(conf_score)
        evaluation, support = finetune_more_sublabels(dataset_1, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args, mode="confidence")
        metrics[i]["FN 05 " + str(conf_score)] = evaluation
        metrics[i]["FN 05 " + str(conf_score)+"_support"] = support
        
        evaluation, support = finetune_more_sublabels(dataset_2, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args, mode="confidence")
        metrics[i]["FN 07 " + str(conf_score)] = evaluation
        metrics[i]["FN 07 " + str(conf_score)+"_support"] = support
        
        evaluation, support = finetune_more_sublabels(dataset_3, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args, mode="confidence")
        metrics[i]["FN 095 " + str(conf_score)] = evaluation
        metrics[i]["FN 095 " + str(conf_score)+"_support"] = support
        
        evaluation, support = finetune_more_sublabels(dataset_4, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args, mode="confidence")
        metrics[i]["no FN " + str(conf_score)] = evaluation
        metrics[i]["no FN " + str(conf_score)+"_support"] = support
        
        metrics_df = pd.DataFrame.from_dict({(i,j): metrics[i][j]
                        for i in metrics.keys() 
                        for j in metrics[i].keys()},
                    orient='columns').transpose()
        metrics_df.to_csv("metrics/stratified_sublabels_FN_mot_test_sets_metrics.csv")
    i = i+1

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


### Choose Option once we have right results on stratified testing

In [4]:
i = 0
label_num = 4
conf_scores = [0.5]

training_args = TrainingArguments("original_forum", evaluation_strategy="epoch")
metrics = {}
for train_idxs, val_idxs in splits:
    fold_dataset = DatasetDict({
        "train":forum.select(train_idxs),
        "validation":forum.select(val_idxs)
    })
    print(i)
    if i > 8:
        metrics_old = pd.read_csv("metrics/stratified_sublabels_additional_tests.csv")
        model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-cased", num_labels=label_num)
        trainer = Trainer(model=model, args=training_args, train_dataset=fold_dataset["train"], eval_dataset=fold_dataset["validation"], compute_metrics=compute_metrics)
        trainer.train()
        metrics[i] = {}
        trainer.save_model("classif/stratified_cv_sublabels")
        metrics[i]["orig"] = compute_test_metrics(trainer.predict(fold_dataset["validation"]), None)
        metrics[i]["train_support"] = compute_test_metrics(trainer.predict(fold_dataset["train"]), None)
        model = AutoModelForSequenceClassification.from_pretrained("classif/stratified_cv_sublabels", num_labels=label_num)
        pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)
        #dataset_1 = predict_sublabels_with_confidence("mot_05.csv", pipe)
        dataset_2 = predict_sublabels_with_confidence("mot_07.csv", pipe)
        '''dataset_3 = predict_sublabels_with_confidence("mot_095.csv", pipe)
        dataset_4 = predict_sublabels_with_confidence("new_data_both.csv", pipe)'''

        for conf_score in conf_scores:
            print(conf_score)
            '''evaluation, support = finetune_more_sublabels(dataset_1, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args)
            metrics[i]["FN 05 " + str(conf_score) + "_only_minority"] = evaluation
            metrics[i]["FN 05 " + str(conf_score)+"_support_only_minority"] = support'''

            evaluation, support = finetune_more_sublabels(dataset_2, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args)
            metrics[i]["FN 07 " + str(conf_score)+ "_only_minority"] = evaluation
            metrics[i]["FN 07 " + str(conf_score)+"_support_only_minority"] = support

            '''evaluation, support = finetune_more_sublabels(dataset_3, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args)
            metrics[i]["FN 095 " + str(conf_score) + "_only_minority"] = evaluation
            metrics[i]["FN 095 " + str(conf_score)+"_support_only_minority"] = support

            evaluation, support = finetune_more_sublabels(dataset_4, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args)
            metrics[i]["no FN " + str(conf_score)+ "_only_minority"] = evaluation
            metrics[i]["no FN " + str(conf_score)+"_support_only_minority"] = support'''

            metrics_df = pd.DataFrame.from_dict({(i,j): metrics[i][j]
                            for i in metrics.keys() 
                            for j in metrics[i].keys()},
                        orient='columns').transpose()
            metrics_df = pd.concat([metrics_old, metrics_df], ignore_index=True)
            metrics_df.to_csv("metrics/stratified_sublabels_additional_tests.csv")

            '''evaluation, support = finetune_more_sublabels(dataset_1, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args, mode="equal")
            metrics[i]["FN 05 " + str(conf_score) + "_equal"] = evaluation
            metrics[i]["FN 05 " + str(conf_score)+"_support_equal"] = support'''

            evaluation, support = finetune_more_sublabels(dataset_2, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args, mode="equal")
            metrics[i]["FN 07 " + str(conf_score)+ "_equal"] = evaluation
            metrics[i]["FN 07 " + str(conf_score)+"_support_equal"] = support

            '''evaluation, support = finetune_more_sublabels(dataset_3, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args, mode="equal")
            metrics[i]["FN 095 " + str(conf_score) + "_equal"] = evaluation
            metrics[i]["FN 095 " + str(conf_score)+"_support_equal"] = support

            evaluation, support = finetune_more_sublabels(dataset_3, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args, mode="equal")
            metrics[i]["no FN " + str(conf_score)+ "_equal"] = evaluation
            metrics[i]["no FN " + str(conf_score)+"_support_equal"] = support'''

            metrics_df = pd.DataFrame.from_dict({(i,j): metrics[i][j]
                            for i in metrics.keys() 
                            for j in metrics[i].keys()},
                        orient='columns').transpose()
            #metrics_df.head()
            metrics_df = pd.concat([metrics_old, metrics_df], ignore_index=True)
            metrics_df.to_csv("metrics/stratified_sublabels_additional_tests.csv")

        for conf_score in conf_scores_lower:
            print(conf_score)

            '''evaluation, support = finetune_more_sublabels(dataset_1, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args, mode="confidence")
            metrics[i]["FN 05 " + str(conf_score) + "_low_conf_minority"] = evaluation
            metrics[i]["FN 05 " + str(conf_score)+"_support_low_conf_minority"] = support'''

            evaluation, support = finetune_more_sublabels(dataset_2, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args, mode="confidence")
            metrics[i]["FN 07 " + str(conf_score)+ "_low_conf_minority"] = evaluation
            metrics[i]["FN 07 " + str(conf_score)+"_support_low_conf_minority"] = support

            '''evaluation, support = finetune_more_sublabels(dataset_3, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args, mode="confidence")
            metrics[i]["FN 095 " + str(conf_score) + "_low_conf_minority"] = evaluation
            metrics[i]["FN 095 " + str(conf_score)+"_support_low_conf_minority"] = support

            evaluation, support = finetune_more_sublabels(dataset_3, conf_score, fold_dataset["train"], label_num, fold_dataset["validation"], training_args, mode="confidence")
            metrics[i]["no FN " + str(conf_score)+ "_low_conf_minority"] = evaluation
            metrics[i]["no FN " + str(conf_score)+"_support_low_conf_minority"] = support'''

            metrics_df = pd.DataFrame.from_dict({(i,j): metrics[i][j]
                            for i in metrics.keys() 
                            for j in metrics[i].keys()},
                        orient='columns').transpose()
            #metrics_df.head()
            metrics_df = pd.concat([metrics_old, metrics_df], ignore_index=True)
            metrics_df.to_csv("metrics/stratified_sublabels_additional_tests.csv")
    i = i+1

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


0
1
2
3
4
5
6
7
8
9


Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoi

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Mcc
1,No log,0.508733,0.821577,0.737449,0.750796,0.727741,0.628100
2,No log,0.518351,0.821577,0.747817,0.749376,0.752980,0.632009
3,No log,0.701555,0.821577,0.741729,0.751916,0.732805,0.630653


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Sentence, nonfunction_Sentence, non_noun, non_noun_verb_adj, non_verb_aux, non_verb, function_Sentence, non_noun_adj. If Sentence, nonfunction_Sentence, non_noun, non_noun_verb_adj, non_verb_aux, non_verb, function_Sentence, non_noun_adj are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 241
  Batch size = 16
/opt/conda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Sentence, nonfunction_Sentenc

...

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file config.json from cache at /home/jovyan/.cache/huggingface/hub/models--bert-base-german-cased/snapshots/702774c02b32a4f360d5fea60ab034d64bf0141c/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-german-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 300

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Mcc
1,No log,0.761236,0.751037,0.483567,0.858262,0.451472,0.413138
2,No log,0.607395,0.788382,0.679367,0.685910,0.686783,0.565706
3,No log,0.716195,0.796680,0.697910,0.714232,0.684790,0.578316


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Sentence, nonfunction_Sentence, non_noun, non_noun_verb_adj, non_verb_aux, non_verb, function_Sentence, non_noun_adj. If Sentence, nonfunction_Sentence, non_noun, non_noun_verb_adj, non_verb_aux, non_verb, function_Sentence, non_noun_adj are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 241
  Batch size = 16
/opt/conda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Sentence, nonfunction_Sentenc

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: 0, Sentence, nonfunction_Sentence, non_noun, non_noun_verb_adj, non_verb_aux, 2, function_Sentence, 3, non_verb, 1, non_noun_adj. If 0, Sentence, nonfunction_Sentence, non_noun, non_noun_verb_adj, non_verb_aux, 2, function_Sentence, 3, non_verb, 1, non_noun_adj are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2393
  Batch size = 16
/opt/conda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [12]:
forum_trains[i].shape

(3333, 12)

In [13]:
predict.shape

(992, 4)

In [ ]:
for feature in features:
    print(feature)
    forum_vals, forum_trains = load_and_tokenize_training_set("../Ablation_Studies/forum/valence/train_valence_preproc.csv")
    training_args = TrainingArguments("original_forum", evaluation_strategy="epoch")
    metrics = {}
    for i in range(1):
        print(i)
        model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-cased", num_labels=label_num)
        trainer = Trainer(model=model, args=training_args, train_dataset=forum_trains[i], eval_dataset=forum_vals[i], compute_metrics=compute_metrics)
        trainer.train()
        metrics[i] = trainer.evaluate()
        dataset_1 = load_dataset("csv", data_files="mot_05.csv", split="train")
        dataset_2 = load_dataset("csv", data_files="mot_07.csv", split="train")
        dataset_3 = load_dataset("csv", data_files="mot_095.csv", split="train")
        #dataset_4 = load_dataset("csv", data_files="new_data_both.csv", split="train")
        pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)
        label_0_scores = []
        label_1_scores = []
        #label_2_scores = []
        #label_3_scores = []
        for out in pipe(KeyDataset(dataset_1, "split"), batch_size=8, truncation="only_first"):
            label_0_scores.append(out[0]['score'])
            label_1_scores.append(out[1]['score'])
           # label_2_scores.append(out[2]['score'])
           # label_3_scores.append(out[3]['score'])
        pred_train_df_1 = pd.DataFrame({"split": dataset_1[:]['split'], "0":label_0_scores, "1": label_1_scores#, "2": label_2_scores, "3": label_3_scores 
                                       })
        label_0_scores = []
        label_1_scores = []
       # label_2_scores = []
       # label_3_scores = []
        for out in pipe(KeyDataset(dataset_2, "split"), batch_size=8, truncation="only_first"):
            label_0_scores.append(out[0]['score'])
            label_1_scores.append(out[1]['score'])
            #label_2_scores.append(out[2]['score'])
            #label_3_scores.append(out[3]['score'])
        pred_train_df_2 = pd.DataFrame({"split": dataset_2[:]['split'], "0":label_0_scores, "1": label_1_scores#, "2": label_2_scores, "3": label_3_scores 
                                       })
        label_0_scores = []
        label_1_scores = []
        #label_2_scores = []
        #label_3_scores = []
        for out in pipe(KeyDataset(dataset_3, "split"), batch_size=8, truncation="only_first"):
            label_0_scores.append(out[0]['score'])
            label_1_scores.append(out[1]['score'])
           # label_2_scores.append(out[2]['score'])
           # label_3_scores.append(out[3]['score'])
        pred_train_df_3 = pd.DataFrame({"split": dataset_3[:]['split'], "0":label_0_scores, "1": label_1_scores#, "2": label_2_scores, "3": label_3_scores  
                                       })
        '''label_0_scores = []
        label_1_scores = []
        label_2_scores = []
        label_3_scores = []
        for out in pipe(KeyDataset(dataset_4, "split"), batch_size=8, truncation="only_first"):
            label_0_scores.append(out[0]['score'])
            label_1_scores.append(out[1]['score'])
            label_2_scores.append(out[2]['score'])
            label_3_scores.append(out[3]['score'])
        pred_train_df_4 = pd.DataFrame({"split": dataset_4[:]['split'], "0":label_0_scores, "1": label_1_scores, "2": label_2_scores, "3": label_3_scores  })'''
        for conf_score in conf_scores:
            print(conf_score)
            _1 = pred_train_df_1[pred_train_df_1["0"] > conf_score].copy()
            a_1 = pred_train_df_1[pred_train_df_1["1"] > conf_score].copy()
            #d_1 = pred_train_df_1[pred_train_df_1["2"] > conf_score].copy()
            #n_1 = pred_train_df_1[pred_train_df_1["3"] > conf_score].copy()
            _1["labels"] = 0
            a_1["labels"] = 1
           # d_1["labels"] = 2
           # n_1["labels"] = 3
            predict_1 = pd.concat([_1,a_1#, d_1, n_1
                                  ]).sample(frac=1)
            predict_1.rename(columns={"split":"Sentence"}, inplace=True)
            predict_1.to_csv("enhanced_test_1.csv")
            enhanced_pred_1 = load_tokenize_enhanced_set("enhanced_test_1.csv")
            enhanced_1 = concatenate_datasets([forum_trains[i], enhanced_pred_1])
            enhanced_1 = enhanced_1.shuffle()
            model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-cased", num_labels=label_num)
            trainer = Trainer(model=model, args=training_args, train_dataset=enhanced_1, eval_dataset=forum_vals[i], compute_metrics=compute_metrics)
            trainer.train()
            metrics[i]["FN 05 " + str(conf_score)] = trainer.evaluate()
            trainer.save_model("classif/valence_FN_05_"+str(conf_score)+"_model")
            _2 = pred_train_df_2[pred_train_df_2["0"] > conf_score].copy()
            a_2 = pred_train_df_2[pred_train_df_2["1"] > conf_score].copy()
            #d_2 = pred_train_df_2[pred_train_df_2["2"] > conf_score].copy()
            #n_2 = pred_train_df_2[pred_train_df_2["3"] > conf_score].copy()
            _2["labels"] = 0
            a_2["labels"] = 1
           # d_2["labels"] = 2
           # n_2["labels"] = 3
            predict_2 = pd.concat([_2,a_2#, d_2, n_2
                                  ]).sample(frac=1)
            predict_2.rename(columns={"split":"Sentence"}, inplace=True)
            predict_2.to_csv("enhanced_test_2.csv")
            enhanced_pred_2 = load_tokenize_enhanced_set("enhanced_test_2.csv")
            enhanced_2 = concatenate_datasets([forum_trains[i], enhanced_pred_2])
            enhanced_2 = enhanced_2.shuffle()
            model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-cased", num_labels=label_num)
            trainer = Trainer(model=model, args=training_args, train_dataset=enhanced_2, eval_dataset=forum_vals[i], compute_metrics=compute_metrics)
            trainer.train()
            metrics[i]["FN 07 " + str(conf_score)] = trainer.evaluate()
            trainer.save_model("classif/valence_FN_07_"+str(conf_score)+"_model")
            _3 = pred_train_df_3[pred_train_df_3["0"] > conf_score].copy()
            a_3 = pred_train_df_3[pred_train_df_3["1"] > conf_score].copy()
           # d_3 = pred_train_df_3[pred_train_df_3["2"] > conf_score].copy()
           # n_3 = pred_train_df_3[pred_train_df_3["3"] > conf_score].copy()
            _3["labels"] = 0
            a_3["labels"] = 1
           # d_3["labels"] = 2
           # n_3["labels"] = 3
            predict_3 = pd.concat([_3,a_3#, d_3, n_3
                                  ]).sample(frac=1)
            predict_3.rename(columns={"split":"Sentence"}, inplace=True)
            predict_3.to_csv("enhanced_test_3.csv")
            enhanced_pred_3 = load_tokenize_enhanced_set("enhanced_test_3.csv")
            enhanced_3 = concatenate_datasets([forum_trains[i], enhanced_pred_3])
            enhanced_3 = enhanced_3.shuffle()
            model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-cased", num_labels=label_num)
            trainer = Trainer(model=model, args=training_args, train_dataset=enhanced_3, eval_dataset=forum_vals[i], compute_metrics=compute_metrics)
            trainer.train()
            metrics[i]["FN 095 " + str(conf_score)] = trainer.evaluate()
            '''_4 = pred_train_df_4[pred_train_df_4["0"] > conf_score].copy()
            a_4 = pred_train_df_4[pred_train_df_4["1"] > conf_score].copy()
            d_4 = pred_train_df_4[pred_train_df_4["2"] > conf_score].copy()
            n_4 = pred_train_df_4[pred_train_df_4["3"] > conf_score].copy()
            _4["labels"] = 0
            a_4["labels"] = 1
            d_4["labels"] = 2
            n_4["labels"] = 3
            predict_4 = pd.concat([_4,a_4, d_4, n_4]).sample(frac=1)
            predict_4.rename(columns={"split":"Sentence"}, inplace=True)
            predict_4.to_csv("enhanced_test_4.csv")
            enhanced_pred_4 = load_tokenize_enhanced_set("enhanced_test_4.csv")
            enhanced_4 = concatenate_datasets([forum_trains[i], enhanced_pred_4])
            enhanced_4 = enhanced_4.shuffle()
            model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-cased", num_labels=label_num)
            trainer = Trainer(model=model, args=training_args, train_dataset=enhanced_4, eval_dataset=forum_vals[i], compute_metrics=compute_metrics)
            trainer.train()
            metrics[i]["no FN " + str(conf_score)] = trainer.evaluate()'''
            trainer.save_model("classif/valence_FN_095_"+str(conf_score)+"_model")
            metrics_df = pd.DataFrame.from_dict({(i,j): metrics[i][j]
                               for i in metrics.keys() 
                               for j in metrics[i].keys()},
                           orient='columns').transpose()
        #metrics_df.head()
            metrics_df.to_csv("metrics/valence_FN_mot_test_sets_metrics_average_none.csv")